# Dimensionality reduction
**V.0.1 - Alpha testing, [contributions](#contributions)**

fMRI data often has a dimensionality problem: we get approximately 100,000 voxels (i.e., features) per volume, but only 100s of time points or trials (i.e., examples). This makes it very hard for machine learning algorithms to model how each voxel contributes. For more general information on this problem, also dubbed the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality), see [these slides from the Texas A&M University Computer Science and Engineering Department](http://courses.cs.tamu.edu/choe/11spring/633/lectures/slide08.pdf). For a neuroimaging-specific view on the curse of dimensionality, you might want to take a look at [Mwangi et al.'s Neuroinformatics review from 2014](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4040248/).

## Goal of this script
1. Learn to compute the covariance of a dataset.  
2. Reduce the feature space using principal component analysis (PCA).
3. Reduce the feature space using indipendent component analysis (ICA).
4. Reduce the feature space using feature selection techniques.  
5. Run classification on the dimensionality reduced feature space.  

**Recap:** The localizer data we are working with ([Kim et al., 2017](https://doi.org/10.1523/JNEUROSCI.3272-16.2017)) consists of 3 runs with 5 blocks for each category. In the matlab stimulus file, the first row has the stimulus labels for the 1st, 2nd and 3rd runs of the localizer. Each run was 310 TRs.
The 4th row contains the time when the stimulus was presented for each of the runs. The stimulus labels and their corresponding categories are as follows:  
1 = Faces, 2 = Scenes, 3 = Objects

## Table of Contents
[1. Preprocessing](#preprocessing)  

[2. Covariance](#covariance)  

[3. PCA](#pca)  
>[3.1 Plot PCA](#plot_pca)  

[4. ICA](#ica)  
>[3.1 Plot ICA](#plot_ica)  

[5. Dimensionality Reduction](#dim_red)  
>[5.1 Feature Selection: ROI](#roi)  
>[5.2 Feature Selection: Top N voxels](#top_n)  
>[5.3 Feature Selection: Variance Threshold](#var_thresh)  
>[5.4 Feature Selection: Univariate](#univariate)  
>[5.5 Feature Selection: Recursive Feature Elimination](#rfe)  

Exercises
>[Exercise 1](#ex1)  
>[Exercise 2](#ex2)  
>[Exercise 3](#ex3)  
>[Exercise 4](#ex4)  
>[Exercise 5](#ex5)  
>[Exercise 6](#ex6)  
>[Exercise 7](#ex7)  
>[Exercise 8](#ex8)  
>[Exercise 9](#ex9)  
>[Exercise 10](#ex10)  


In [1]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
# Import neuroimaging, analysis and general libraries
import nibabel as nib
import numpy as np
import scipy.io
from scipy import stats
from time import time
import pandas as pd

# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns 
# %matplotlib notebook

# Machine learning libraries
from nilearn.input_data import NiftiMasker
from nilearn.masking import compute_epi_mask
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit 
from sklearn.svm import SVC
from sklearn.decomposition import PCA, FastICA
from sklearn.feature_selection import SelectKBest, VarianceThreshold, RFECV, chi2, SelectPercentile
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [2]:
from qutils import vdc_data_dir, vdc_all_ROIs, vdc_label_dict, vdc_n_runs

%matplotlib inline 
%autosave 5
sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})

print('data dir = %s' % (vdc_data_dir))
print('ROIs = %s' % (vdc_all_ROIs))
print('Labels = %s' % (vdc_label_dict))
print('number of runs = %s' % (vdc_n_runs))

Autosaving every 5 seconds
data dir = /home/pytorch51/public_FMRI/vdc/
ROIs = ['FFA', 'PPA']
Labels = {1: 'Faces', 2: 'Scenes', 3: 'Objects'}
number of runs = 3


## 1. Preprocessing <a id="preprocessing"></a>

Let's set up some functions that we will be using in this script. 

In [3]:
# Make a function to load the mask data
def load_data(directory, subject_name, mask_name='', num_runs=3, zscore_data=False):
    
    maskdir = (directory + subject_name + "/preprocessed/masks/")

    # Cycle through the masks
    print ("Processing Start ...")
    
    # If there is a mask supplied then load it now
    if mask_name is not '':
        maskfile = (maskdir + "%s_ventral_%s_locColl_to_epi1.nii.gz" % (subject_name, mask_name))

        mask = nib.load(maskfile)
        print ("Loaded %s mask" % (mask_name))

    # Cycle through the runs
    for run in range(1, num_runs + 1):
        epi_in = (directory + subject_name + "/preprocessed/loc/%s_filtered2_d1_firstExampleFunc_r%d.nii" % (subject_name, run))
        print("\t" + epi_in)

        # Load in the fmri data
        epi_data = nib.load(epi_in)
        
        # Mask the data if necessary
        if mask_name is not '':
            nifti_masker = NiftiMasker(mask_img=mask)
            epi_mask_data = nifti_masker.fit_transform(epi_data);
            epi_mask_data = np.transpose(epi_mask_data)
        else:
            # Do a whole brain mask 
            if run == 1:
                mask = compute_epi_mask(epi_data).get_data()  # Compute mask from epi
            else:
                mask *= compute_epi_mask(epi_data).get_data()  # Get the intersection mask (set voxels that are within the mask on all runs to 1, set all other voxels to 0)   
            
            # Reshape all of the data from 4D (X*Y*Z*time) to 2D (voxel*time): not great for memory
            epi_mask_data = epi_data.get_data().reshape(mask.shape[0] * mask.shape[1] * mask.shape[2], epi_data.shape[3])

        # Transpose and z-score (standardize) the data  
        if zscore_data == True:
            scaler = preprocessing.StandardScaler().fit(epi_mask_data)
            preprocessed_data = scaler.transform(epi_mask_data)
        else:
            preprocessed_data = epi_mask_data
        
        # Concatenate the data
        if run == 1:
            concatenated_data = preprocessed_data
        else:
            concatenated_data = np.hstack((concatenated_data, preprocessed_data))
    
    # Apply the whole-brain masking: First, reshape the mask from 3D (X*Y*Z) to 1D (voxel). 
    # Second, get indices of non-zero voxels, i.e. voxels inside the mask. 
    # Third, zero out all of the voxels outside of the mask.
    if mask_name is '':
        mask_vector = np.nonzero(mask.reshape(mask.shape[0] * mask.shape[1] * mask.shape[2], ))[0]
        concatenated_data = concatenated_data[mask_vector, :]
        
    # Return the list of mask data
    return concatenated_data

# Make a function for loading in the labels
def load_labels(directory, subject_name):
    stim_label = [];
    stim_label_concatenated = [];
    for run in range(1,4):
        in_file= (directory + subject_name + '/ses-day2/design_matrix/' + "%s_localizer_0%d.mat" % (subject_name, run))

        # Load in data from matlab
        stim_label = scipy.io.loadmat(in_file);
        stim_label = np.array(stim_label['data']);

        # Store the data
        if run == 1:
            stim_label_concatenated = stim_label;
        else:       
            stim_label_concatenated = np.hstack((stim_label_concatenated, stim_label))

    print("Loaded ", subject_name)
    return stim_label_concatenated


# Convert the TR
def label2TR(stim_labels, num_runs, TR, TRs_run):
    
    # Calculate the number of events/run
    _, events = stim_labels.shape
    events_run = int(events / num_runs)
    
    # Preset the array with zeros
    stim_label_TR = np.zeros((TRs_run * 3, 1))

    # Cycle through the runs
    for run in range(0, num_runs):

        # Cycle through each element in a run
        for i in range(events_run):

            # What element in the concatenated timing file are we accessing
            time_idx = run * (events_run) + i

            # What is the time stamp
            time = stim_labels[2, time_idx]

            # What TR does this timepoint refer to?
            TR_idx = int(time / TR) + (run * (TRs_run - 1))

            # Add the condition label to this timepoint
            stim_label_TR[TR_idx]=stim_label_allruns[0, time_idx]
        
    return stim_label_TR

# Create a function to shift the size
def shift_timing(label_TR, TR_shift_size):
    
    # Create a short vector of extra zeros
    zero_shift = np.zeros((TR_shift_size, 1))

    # Zero pad the column from the top
    label_TR_shifted = np.vstack((zero_shift, label_TR))

    # Don't include the last rows that have been shifted out of the time line
    label_TR_shifted = label_TR_shifted[0:label_TR.shape[0],0]
    
    return label_TR_shifted


# Extract bold data for non-zero labels
def reshape_data(label_TR_shifted, masked_data_all):
    label_index = np.nonzero(label_TR_shifted)
    label_index = np.squeeze(label_index)
    
    # Pull out the indexes
    indexed_data = np.transpose(masked_data_all[:,label_index])
    nonzero_labels = label_TR_shifted[label_index] 
    
    return indexed_data, nonzero_labels

# Run a basic n fold classification
def classification(classifier, data, labels, n_folds=3, test_size=0.1):
    
    # How many folds of the classifier
    skfold = StratifiedShuffleSplit(n_splits=n_folds, test_size=test_size) 

    clf_score = np.array([])
    for train, test in skfold.split(data, labels):

        # Pull out the sample data
        train_data = data[train, :]
        test_data = data[test, :]
        
        # Train and test the classifier
        clf = classifier.fit(train_data, labels[train])
        clf_score = np.hstack((clf_score, clf.score(test_data, labels[test])))

    return clf_score.mean()


**Self-study:** We use the method .get_data() on Nifti files in the load_data above. Find out why we need this method or what this method does in the [NiBabel Documentation](http://nipy.org/nibabel/gettingstarted.html).

The way we analyzed the data in week 3 was problematic because it assumed that each observation was independent. However, note that each trial within a block occurred within 1.5s of each other. In other words, there was substantial overlap in the HRF between adjacent trials. The autocorrelation in fMRI data means that adjacent time points will be similar despite potential variability in the underlying signal.

One way to appropriately deal with this problem of independence is to treat blocks, rather than stimulus events, as our inputs in these analyses.

**Exercise 1:** <a id="ex1"></a> In the cell below, finish making a function that converts bold_data (the output of reshape_data) and labels into blockwise data, rather than eventwise data.

In [ ]:
# Take in a brain volume and label vector that is the length of the event number and convert it into a list the length of the block number
def blockwise_sampling(eventwise_data, eventwise_labels, events_per_block=10):
    
    # How many events are expected
    expected_blocks = int(eventwise_data.shape[0] / events_per_block)
    
    # Average the BOLD data for each block of trials
    blockwise_data = np.zeros((expected_blocks, eventwise_data.shape[1]))
    for block in list(range(expected_blocks)):
        blockwise_data[block, :] = np.mean(eventwise_data[block * events_per_block : (block + 1) * events_per_block, :], axis=0)
    
    # Down sample labels
    blockwise_labels = eventwise_labels[::events_per_block]
    
    # Check labels were correctly down sampled
    
    # Report the new variable sizes
    print('Expected blocks: %d; Resampled blocks: %d' % (expected_blocks, blockwise_data.shape[0]))
    
    # Return the variables downsampled_data and downsampled_labels
    return blockwise_data, blockwise_labels

In [ ]:
directory = vdc_data_dir
subject_name = 'sub-01'
num_runs = vdc_n_runs

bold_data = load_data(directory, subject_name, mask_name='', num_runs=3, zscore_data=False).T
labels = load_labels(directory, subject_name)

blockwise_data, blockwise_labels = blockwise_sampling(bold_data, labels, events_per_block=10)

print('shape - bold data:', np.shape(bold_data))
print('shape - labels:',np.shape(labels))
print('shape - blockwise_data:',np.shape(blockwise_data))
print('shape - blockwise_labels:',np.shape(blockwise_labels))

Processing Start ...
	/home/pytorch51/public_FMRI/vdc/sub-01/preprocessed/loc/sub-01_filtered2_d1_firstExampleFunc_r1.nii
	/home/pytorch51/public_FMRI/vdc/sub-01/preprocessed/loc/sub-01_filtered2_d1_firstExampleFunc_r2.nii


In [ ]:
eventwise_data, eventwise_labels = bold_data, labels
events_per_block=10

# How many events are expected
expected_blocks = int(eventwise_data.shape[0] / events_per_block)

# Average the BOLD data for each block of trials into blockwise_data
# Downsample labels into blockwise_labels
n_events, n_voxels = np.shape(eventwise_data)
boldwise_data = np.zeros((expected_blocks, n_voxels))
for t in range(expected_blocks):
    t_start, t_end = t * events_per_block, (t + 1) * events_per_block
    boldwise_data[t,:] = np.mean(eventwise_data[t_start:t_end, :], axis = 0)

# Report the new variable sizes
print('Expected blocks: %d; Resampled blocks: %d' % (expected_blocks, blockwise_data.shape[0]))

**Self-study:** We introduce a simple kind of debugging here, as we print both the number of expected and resampled blocks. Thus, if something went wrong, we would be able to spot it the output. Learn about more ways of debugging your code, e.g. using [if statements](https://stackoverflow.com/questions/5142418/what-is-the-use-of-assert-in-python) and [assertion statements](https://wiki.python.org/moin/UsingAssertionsEffectively).

## 2. Covariance <a id="covariance"></a>

As a precursor to understanding dimensionality reduction techniques, we need to learn how to compute the covariance matrix because it is often used in these methods.  

We are going to work with whole-brain data this time. You might have memory issues but this is an important problem to grapple with. There are nearly 1 million voxels in every volume we acquire, of which about 15% are in the brain. Despite >100,000 voxels, we only have <1000 time points.

In [ ]:
# Preset variables
# Make sure you edit the following line to reflect the directory where you are accessing the VDC dataset
# dir = '/gpfs/milgram/data/cmhn-s18/datasets/vdc/' #Yale
# dir = '/jukebox/pniintel/brainiak_edu/datasets/vdc/' #Princeton
# dir ='/home/NEU350/datasets/vdc/'
print(vdc_data_dir)

num_runs=3
TR=1.5
hrf_lag = 4.5  # In seconds what is the lag between a stimulus onset and the peak bold response
shift_size = int(hrf_lag / TR)  # Convert the shift into TRs

sub_id = 1

# Convert the number into a participant folder name
if (sub_id < 10):
    sids = '0' + str(sub_id)
else:
    sids = str(sub_id)   

# Specify the subject name
sub = 'sub-' + sids

# Load subject labels
stim_label_allruns = load_labels(vdc_data_dir, sub)

# Load the fMRI data
epi_mask_data_all = load_data(vdc_data_dir, sub, mask_name='')

# This can differ per participant
print(sub, '= TRs: ', epi_mask_data_all.shape[1], '; Voxels: ', epi_mask_data_all.shape[0])
TRs_run = int(epi_mask_data_all.shape[1] / num_runs)

# Convert the timing into TR indexes
stim_label_TR = label2TR(stim_label_allruns, num_runs, TR, TRs_run)

# Shift the data some amount
stim_label_TR_shifted = shift_timing(stim_label_TR, shift_size)

# Perform the reshaping of the data
bold_data, labels = reshape_data(stim_label_TR_shifted, epi_mask_data_all)

# Down sample the data to be blockwise rather than trialwise
bold_data, labels = blockwise_sampling(bold_data, labels)

The covariance of two variables is calculated as follows: $$ Cov(X,Y) = \frac{\sum_{1}^{N}(X-\bar{X})(Y-\bar{Y})}{(N-1)}$$
where $\mbox{  }  \bar{X} = mean(X), \mbox{  } \bar{Y} = mean(Y), \mbox{  } N = \mbox{number of samples } $

In fMRI, X and Y could be time-series data for two voxels in the simplest case. We could extend this analysis to creating a covariance matrix for a set of voxels.

**Exercise 2:** <a id="ex2"></a> Compute the covariance

In [ ]:
# Enter your code here

# Compute the mean of one column of the bold data X

# Compute the mean of any other column of the bold data  Y

# Compute the differences from the mean for X and Y.

# Compute the summed product of the differences.

# Compute the covariance


# Compare your result to the answer got by using np.cov(X,Y)


The covariance is dependent on the units of the measurement. Its value is thus not easily interpretable or comparable across datasets -- e.g. is there a strong relationship between X and Y if the covariance is 200 as compared to 2 or 2000?

Correlation solves this problem by normalizing the range of the covariance from -1 to +1.

$$ Corr(X,Y) = \frac{Cov(X,Y)}{\sqrt{\frac{\sum_{1}^{N}(X-\bar{X})^2}{(N-1)}}\sqrt{\frac{\sum_{1}^{N}(Y-\bar{Y})^2}{(N-1)}}}$$

**Exercise 3:** <a id="ex3"></a> Compute the correlation manually and with a pre-specified function 

In [ ]:
# Compute the correlation manually

# Now with a function  


**Exercise 4**: <a id="ex4"></a> np.cov can take a matrix as input and calculates the covariance matrix of all columns. Extend the **covariance** computation to a group of 100 voxels in order to make a voxel by voxel covariance matrix. 

In [ ]:
# Insert your code here.

# Subselect 100 voxels from bold_data into a matrix.

# Use np.cov() to comput the covariance of this matrix.

**Bonus:**  What is the maximum number of columns/voxels in the present dataset from which you could calculate the covariance matrix without exceeding available memory (currently 12GB)? Solve this analytically rather than by trial and error.


## 3. PCA <a id="pca"></a>

We will use PCA (principal component analysis) to **reduce the dimensionality** of the data. Some voxels may contain correlated information or no information and so the original voxel-dimensional data can be projected into a lower-dimensional "component" space without losing much information.

![image](https://cdn-images-1.medium.com/max/1200/1*Iri_LDMXuz2Qac-8KPeESA.png)

In [ ]:
# We now use the PCA function in scikit-learn to reduce the dimensionality of the data
# The number of components was chosen arbitrarily and is likely too high
pca = PCA(n_components=20)
bold_pca = pca.fit_transform(bold_data)

print(bold_data.shape)
print(bold_pca.shape)

### 3.1 Plot the PCA dataset for one subject <a id="plot_pca"></a>

Let's visualize the variance in the data along different component dimensions. Dimension 1 must display the largest variation and the later dimensions should start clumping the data.

In [ ]:
# Setting plotting parameter
b=75

# Plot
n_plots = 4
components_to_plot = [0,1,2,19]

f, axes = plt.subplots(1, n_plots, figsize=(14, 14/n_plots))

for i in range(n_plots): 
    axes[i].hist(bold_pca[:, components_to_plot[i]], bins=b)
    # mark the plots 
    axes[i].set_title('PC Dimension %d'%(components_to_plot[i]+1))
    axes[i].set_ylabel('Frequency')
    axes[i].set_xlabel('Value')    
    axes[i].set_xticks([])
    axes[i].set_yticks([])    

f.tight_layout()

Let's visualize the relationship between variances across pairs of components.

In [ ]:
# Setting plotting parameters
alpha_val = .7

# Plot
n_plots = 3 
f, axes = plt.subplots(1, n_plots, figsize=(14,5))

axes[0].scatter(bold_pca[:,0], bold_pca[:,1], alpha=alpha_val, marker='.')
axes[0].set_title('PCA Dimensions\n1 x 2')

axes[1].scatter(bold_pca[:,0],bold_pca[:,2], alpha=alpha_val, marker='.') 
axes[1].set_title('PCA Dimensions\n1 x 3')

axes[2].scatter(bold_pca[:,18],bold_pca[:,19], alpha=alpha_val, marker='.') 
axes[2].set_title('PCA Dimensions\n18 x 19')

for i in range(n_plots): 
    axes[i].axis('equal')
    axes[i].set_xticks([])
    axes[i].set_yticks([])    
f.tight_layout()

**Self-study:** Why are there 3 groups of data in the first two PCA components? Hint: they are not the three conditions!

**Exercise 5:** <a id="ex5"></a> Look up how to display the amount of variance in the original data explained by each component, called a ["scree" plot](https://www.theanalysisfactor.com/factor-analysis-how-many-factors/), [e.g. in this guide](https://www.analyticsvidhya.com/blog/2016/03/practical-guide-principal-component-analysis-python/). Make a scree plot based on the components above. Interpret the plot: How many components would be sufficient to account for most of the variance?

Compare whole-brain classification accuracy without and with PCA dimensionality reduction.

In [ ]:
# Get the baseline, whole brain decoding accuracy.
print('Original size: ', bold_data.shape)
classifier = SVC(kernel="linear", C=0.0001)

start = time()
score_all = classification(classifier, bold_data, labels)
end = time()
print('Accuracy: %0.2f; Run time: %0.2fs' %(score_all, end - start))

In [ ]:
# Run the classifier on the PCA data with different numbers of components included.
print('New size after PCA: ', bold_pca.shape)

start = time()
score_pca = classification(classifier, bold_pca, labels)
end = time()
print('Accuracy: %0.2f; Run time: %0.2fs' %(score_pca, end - start))

**Exercise 6:** <a id="ex6"></a> How does accuracy change when we change the number of components?

## 4. ICA <a id="ica"></a>

PCA is not always the best choice to identify the most explanatory dimensions in our data. If our data is non-Gaussian, Independent Components Analysis (ICA) may be better for dimensionality reduction. Learn more about when to use ICA instead of PCA and how they differ using the [lecture slides form the Center of Aerospace Structures at the University of Colorado Boulder](https://www.colorado.edu/engineering/CAS/courses.d/ASEN6519.d/Lectures.d/Lecture10_11.6519.pdf).

In [ ]:
# Compute ICA
ica = FastICA(n_components=3, whiten=1)  # Set parameters
bold_ica = ica.fit_transform(np.transpose(bold_data))  # Reconstruct signals/sources
mixing_matrix = ica.mixing_  # Get estimated mixing matrix

print(bold_ica.shape)
print(mixing_matrix.shape)  # (n_features, n_components)

### 4.1 Plot the ICA dataset for one subject <a id="plot_ica"></a>

Let's visualize the ICA sources.

In [ ]:
# visualize the ICA sources.

# Setting plotting parameters
a=0.2
size=20

# Plot
plt.subplots(1,3, figsize=(14,14/3))
plt.subplot(1, 3, 1)
plt.title('ICA Source\n1 vs 2')
plt.scatter(bold_ica[:,0], bold_ica[:,1], color='red', alpha=a,  marker='x', s=size)
plt.subplot(1, 3, 2)
plt.title('ICA Source\n1 vs 3')
plt.scatter(bold_ica[:,0],bold_ica[:,2], color='green', alpha=a, marker='d', s=size)
plt.subplot(1, 3, 3)
plt.title('ICA Source\n2 vs 3')
plt.scatter(bold_ica[:,1],bold_ica[:,2], color='blue', alpha=a, marker="*", s=size)
plt.tight_layout()

In [ ]:
# visualize the ICA weights

x=list(range(len(mixing_matrix[:,0])))
plt.gca().set_color_cycle(['red', 'green', 'blue'])
plt.figure()
plt.title('ICA Weights')
plt.plot(x, mixing_matrix[:,0], x, mixing_matrix[:,1], x, mixing_matrix[:,2], linewidth=2)
plt.legend(['Source 1', 'Source 2', 'Source 3'], loc='center left')
plt.xlabel('Time (Block)')
plt.ylabel('Weight')


**Exercise 7:** <a id="ex7"></a> In the above analyses, three sources were modelled. What do you think these sources represent? If you ran this analysis with more than 3 sources, what do you think those additional sources would represent?

**A:**

**Exercise 8:** <a id="ex8"></a> Redo the analysis with data created with a PPA mask and plot the results. Are there any differences between the ICA analyses using an FFA mask and the analyses using a PPA mask?.

In [ ]:
# Insert your code here


## 5. Dimensionality reduction with feature selection <a id="dim_red"></a>

You have used dimensionality reduction techniques to speed up and possibly improve classification. The choice of parameters for the classifiers has been arbitary and the number of dimensions for PCA and ICA was based on your observation. Below we'll create pipeline to optimize these parameters.

### 5.1 Feature Selection: ROI <a id="roi"></a>

In previous classes you have been doing voxel selection, even if you didn't know. Using the FFA, PPA, or any other mask is an example of voxel selection that can greatly improve your decoding accuracy if you have reason to believe that discriminative information is contained in the ROI.

In [ ]:
# Run the classifier on the FFA masked data with blockwise sampling
epi_mask_data_FFA = load_data(vdc_data_dir, sub, mask_name='FFA')
bold_FFA, labels = reshape_data(stim_label_TR_shifted, epi_mask_data_FFA)
bold_FFA, labels = blockwise_sampling(bold_FFA, labels)

start = time()
score_FFA = classification(classifier, bold_FFA, labels)
end = time()
print('Accuracy: %0.2f; Run time: %0.2fs' %(score_FFA, end - start))

**Self-study:** The FFA and PPA were defined using a functional localizer in this dataset. Do you think this presents any concerns?

### 5.2 Feature Selection: Top n Values <a id="top_n"></a>

We have as many features as we have voxels. We can reduce the feature space using metrics of our choice and picking the top n voxels that satisfy a metric. Below, we first standardize the data and take the top n voxels that had the highest mean activation.

In [ ]:
# Compute the 95th percentile for the mean voxel activation across time/blocks to use as a cut-off value
mean_threshold = np.percentile(np.mean(bold_data, axis=0), 95)

# Use the cut-off value to access all voxels with top 5% mean activity
bold_top_N = bold_data[:, mean_threshold <= np.mean(bold_data, axis=0)]

print('Top_N size: ', bold_top_N.shape)

score_top_N = classification(classifier, bold_top_N, labels)

print('Accuracy: %0.2f' %(score_top_N))

**Self-study:** An alternative way to calculate the percentile is to use [SelectPercentile from sklearn.feature_selection](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html). Replicate the above dimensionality reduction procedure using the SelectPercentile method.

### 5.3 Feature Selection: Variance Threshold <a id="var_thresh"></a>

There are also functions built into sci-kit learn that we could use. One measure that we can use to select voxels is variance, as it can be useful to detect and reomve features with relatively low variance. We use the [sklearn VarianceThreshold method](http://scikit-learn.org/stable/modules/feature_selection.html#variance-threshold).

In [ ]:
# Take the 95th percentile of variance
var_threshold = np.percentile(np.std(bold_data, axis=0) ** 2, 95)

selector = VarianceThreshold(threshold=var_threshold)
bold_VT = selector.fit_transform(bold_data)

print('Top variance size: ', bold_VT.shape)

score_VT = classification(classifier, bold_VT, labels)

print('Accuracy: %0.2f' %(score_VT))

### 5.4 Feature Selection: Univariate <a id="univariate"></a>

We can also use a variety of univariate methods to do feature selection using methods built into scikit-learn. We use  [chi-squared values](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2) and the [SelectKBest method](http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection) below.

In [ ]:
bold_chi = SelectKBest(chi2, k=100).fit_transform(bold_data, labels)
print('K top features: ', bold_chi.shape)

score_chi = classification(classifier, bold_chi, labels)
print('Accuracy: %0.2f' %(score_chi))

**Self-Study:** Not sure what is being tested here? Go find out!

**Exercise 9:**<a id="ex9"></a> Apply a dimensionality reduction procedure other than those above that improves performance. What is its run time?

### 5.5 Feature Selection: Recursive Feature Elimination (RFE) <a id="rfe"></a>

This technique eliminates features iteratively based on a measure that we can define, e.g., classification accuracy. At each step of the iteration a certain number of features are eliminated. This is done iteratively until an optimal set of features are found. Importantly, cross-validation is used during each iteration to select the best set of features at each stage. scikit-learn makes this incredibly easy with the [method RFECV](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination).

Because RFE is running many iterations of your classifier, you want to be careful that your data does not have too many features or else it will run for a long time. Also, the number of iterations is   dependent on both the number of features in your data and on how you set the step size of the RFE method. The step size corresponds to the number of features to remove at each iteration. Thus, the number of iterations will roughly be nb_of_features/step_size.

In [ ]:
# The "accuracy" scoring is proportional to the number of correct classifications
rfecv = RFECV(estimator=classifier, 
              step=500, 
              cv=StratifiedShuffleSplit(n_splits=3, test_size=0.1),
              scoring='accuracy')
rfecv.fit(bold_VT, labels)

**Exercise 10**: <a id="ex10"></a> Report and visualize the results of RFE, including the optimal number of features.

**Novel contribution:** <a id="novel"></a> be creative and make one new discovery by adding an analysis, visualization, or optimization.

## Contributions <a id="contributions"></a>

M. Kumar, C. Ellis and N. Turk-Browne produced the initial notebook  
T. Meissner minor edits and added the ICA section  
Q. Lu revise PCA plots 